In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from IPython.display import clear_output
!pip3 install -U lazypredict
!pip3 install -U pandas #Upgrading pandas

clear_output()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split


from lightgbm import LGBMClassifier
import lazypredict
from lazypredict.Supervised import LazyClassifier
from IPython.display import clear_output

import time

from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv("../input/spaceship-titanic/train.csv")
test = pd.read_csv("../input/spaceship-titanic/test.csv")
submission = pd.read_csv("../input/spaceship-titanic/sample_submission.csv")
train.head()

# **Data Preprocessing**

In [ ]:
train.shape

In [ ]:
train.describe()

**Displaying Missing Values Column Wise**

In [ ]:
train.isna().sum().sort_values(ascending=False)

Number of rows with missing data in train dataset

In [ ]:
print(sum(train.isna().sum()))

In [ ]:
train.drop(columns=['PassengerId','Name'],axis=1,inplace=True)
test.drop(columns=['PassengerId','Name'],axis=1,inplace=True)

In [ ]:
train[train['HomePlanet'].isna()]

In [ ]:
df = pd.DataFrame(train.isna().sum().sort_values(ascending=False))
df['columns']=df.index
df['count']=df.iloc[:,:-1]
df.reset_index(drop=True, inplace=True)
df = df.drop(df.columns[[0]],axis = 1)
df

df1 = pd.DataFrame(test.isna().sum().sort_values(ascending=False))
df1['columns']=df1.index
df1['count']=df1.iloc[:,:-1]
df1.reset_index(drop=True, inplace=True)
df1 = df1.drop(df1.columns[[0]],axis = 1)
df1

display(df)
display(df1)

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=True, figsize=(18,6))
fig.suptitle('Null Distribution Column-Wise')
sns.barplot(ax=axes[0],y='columns',x='count',data=df)
sns.barplot(ax=axes[1],y='columns',x='count',data=df1)
axes[0].set_title('Null Distribution in Train Data')
axes[1].set_title('Null Distribution in Test Data')

In [ ]:
plt.figure(figsize=(12,6))
sns.set_theme(style='darkgrid')
sns.countplot(x='HomePlanet',data=train)
plt.show()

In [ ]:
train['Destination'].value_counts()

In [ ]:
df2 = pd.DataFrame(train['Destination'].value_counts())
df2['destination']=df2.index
df2['count']=df2.iloc[:,:-1]
df2.reset_index(drop=True, inplace=True)
df2 = df2.drop(df2.columns[[0]],axis = 1)
df2


In [ ]:
plt.figure(figsize=(19,6))
df2.groupby(['destination']).sum().plot(kind='pie', y='count', autopct='%1.0f%%')
plt.show()

In [ ]:
train.head()

In [ ]:
len(train['Age'].value_counts())

Distribution of People successfully Transported to their Destination based upon Age. We infer the following info
- We find that children/infants were given a lot of preference for their safe Transport to their Destination
- Majority of the people travelling fell in the age range of 20-50

In [ ]:
plt.figure(figsize=(24,6))
sns.countplot(x='Age',hue='Transported',data=train)
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(24,6))
sns.countplot(x='Age',hue='CryoSleep',data=train)
plt.xticks(rotation=90)
plt.show()

**Distribution of Various Amenities Availed by the passenger on the basis of HomePlanet**

In [ ]:
groupby=pd.DataFrame(train.groupby(['HomePlanet']).count()[['VIP','CryoSleep','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Transported','Age']])

In [ ]:
groupby['HomePlanet']=groupby.index
groupby.reset_index(drop=True)

In [ ]:
fig, axes = plt.subplots(3, 3, sharex=True, figsize=(18,10))
fig.suptitle('Distribution of Various Amenities availed based on HomePlanet')
ax=sns.barplot(ax=axes[0,0],y='VIP',x='HomePlanet',data=groupby)
for i in ax.containers:
    ax.bar_label(i,)
ax=sns.barplot(ax=axes[0,1],y='CryoSleep',x='HomePlanet',data=groupby)
for i in ax.containers:
    ax.bar_label(i,)
ax=sns.barplot(ax=axes[0,2],y='RoomService',x='HomePlanet',data=groupby)
for i in ax.containers:
    ax.bar_label(i,)
ax=sns.barplot(ax=axes[1,0],y='FoodCourt',x='HomePlanet',data=groupby)
for i in ax.containers:
    ax.bar_label(i,)
ax=sns.barplot(ax=axes[1,1],y='ShoppingMall',x='HomePlanet',data=groupby)
for i in ax.containers:
    ax.bar_label(i,)
ax=sns.barplot(ax=axes[1,2],y='Spa',x='HomePlanet',data=groupby)
for i in ax.containers:
    ax.bar_label(i,)
ax=sns.barplot(ax=axes[2,0],y='Transported',x='HomePlanet',data=groupby)
for i in ax.containers:
    ax.bar_label(i,)
ax=sns.barplot(ax=axes[2,1],y='Age',x='HomePlanet',data=groupby)
for i in ax.containers:
    ax.bar_label(i,)


In [ ]:
imputer_cols = ["Age","FoodCourt","ShoppingMall","Spa","VRDeck","RoomService"]
imputer = SimpleImputer()
imputer.fit(train[imputer_cols])
train[imputer_cols] = imputer.transform(train[imputer_cols])
test[imputer_cols] = imputer.transform(test[imputer_cols])
train["HomePlanet"].fillna('Z', inplace=True)
test["HomePlanet"].fillna('Z', inplace=True)

In [ ]:
label_cols = ["HomePlanet", "CryoSleep","Cabin", "Destination" ,"VIP"]
def label_encoder(train,test,columns):
    for col in columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
        train[col] = LabelEncoder().fit_transform(train[col])
        test[col] =  LabelEncoder().fit_transform(test[col])
    return train, test

train ,test = label_encoder(train,test ,label_cols)

In [ ]:
train.head()

In [ ]:
train['HomePlanet'].value_counts()

In [ ]:
train.drop(["Cabin"] , axis = 1 ,inplace = True)
test.drop(["Cabin"] , axis = 1 ,inplace = True)

In [ ]:
X = train.drop(columns=['Transported'] , axis =1 )
y = train['Transported']
X_train , X_test , y_train , y_test = train_test_split(X , 
                                                       y, 
                                                       random_state = 12 ,
                                                       test_size =0.33)

In [ ]:
clf = LazyClassifier(verbose=0,
                     ignore_warnings=True,
                     custom_metric=None,
                     predictions=False,
                     random_state=12,
                     classifiers='all')

models, predictions = clf.fit(X_train , X_test , y_train , y_test)
clear_output()

In [ ]:
models = pd.DataFrame(models[:27])

In [ ]:
models['Model']=models.index
models.reset_index(drop=True, inplace=True)

In [ ]:
models

In [ ]:
plt.figure(figsize=(18,6))
sns.lineplot(x="Model", y="Time Taken", data=models)
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(18,6))
sns.lineplot(x="Model", y="Accuracy", data=models)
plt.xticks(rotation=90)
plt.show()

In [ ]:
train.head()
train

In [ ]:
X = train.iloc[:,:-1]
y = train.iloc[:,-1:]

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=32)

In [ ]:
model = LGBMClassifier()
model.fit(X_train, y_train,verbose=0)

In [ ]:
lgb_params = {
    'objective' : 'binary',
    'n_estimators' :50,
    'learning_rate' : 0.08
}

RANDOM_STATE = 12 
FOLDS = 5
STRATEGY = 'median'

lgb_predictions = 0
lgb_scores = []
lgb_fimp = []
LGBM_FEATURES = list(train.columns)[:-1]
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)
for fold, (train_idx, valid_idx) in enumerate(skf.split(train[LGBM_FEATURES], train['Transported'])):
    print(f'\033[94m')
    print(10*"=", f"Fold={fold+1}", 10*"=")
    start_time = time.time()
    
    X_train, X_valid = train.iloc[train_idx][LGBM_FEATURES], train.iloc[valid_idx][LGBM_FEATURES]
    y_train , y_valid = train['Transported'].iloc[train_idx] , train['Transported'].iloc[valid_idx]
    
    model = LGBMClassifier(**lgb_params)
    model.fit(X_train, y_train,verbose=0)
    
    preds_valid = model.predict(X_valid)
    acc = accuracy_score(y_valid,  preds_valid)
    lgb_scores.append(acc)
    run_time = time.time() - start_time
    
    fim = pd.DataFrame(index=LGBM_FEATURES,
                 data=model.feature_importances_,
                 columns=[f'{fold}_importance'])
    lgb_fimp.append(fim)
    
    print(f"Fold={fold+1}, Accuracy score: {acc:.2f}%, Run Time: {run_time:.2f}s")
    test_preds = model.predict(test[LGBM_FEATURES]) 
    lgb_predictions += test_preds/FOLDS
print("")
print("Mean Accuracy :", np.mean(lgb_scores))

In [ ]:
submission['Transported'] = lgb_predictions.astype("bool")
submission.to_csv("submission.csv",index=False)
submission.head()

In [ ]:
models = {"LogisticClassifier":LogisticRegression(solver='liblinear'),
          "KNearest": KNeighborsClassifier(),
          "Support Vector Classifier": SVC(),
          "DecisionTreeClassifier": DecisionTreeClassifier(),
          "XGBClassifier":XGBClassifier(),
          "LightLGBM Classifier":LGBMClassifier(),
          #"CatBoostClassifier":CatBoostClassifier()
          }

In [ ]:
for name, vals in models.items():
    model = vals.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = accuracy_score(y_test,y_pred)
    print(f'{name} accuracy is {score}')

In [ ]:
xgbc = XGBClassifier().fit(X_train, y_train)
y_pred = xgbc.predict(X_test)

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
sub = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')
sub['Transported'] = xgbc.predict(test)
sub.to_csv('submission.csv',index=False)
sub.head()